# Code

In [1]:
# Imports and setup model
import os
import openai
import json
import configobj
import numpy
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import tiktoken
from typing_extensions import Concatenate
from uuid import uuid4
from tqdm.auto import tqdm
import pinecone
from time import sleep
from IPython.display import Markdown

/home/will/anaconda3/envs/thesis/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [28]:
# Setup env vars and constants
config = configobj.ConfigObj('.env')
openai.api_key = config["OPENAI_API_KEY"]
pinecone_key = config["PINECONE_API_KEY"]
pinecone_env = config["PINECONE_ENVIRONMENT"]

embedding_model = "text-embedding-ada-002" 
dimensions = 1536
docs_dir = 'docs/'

In [35]:
# load docs
texts = []

for doc in os.listdir(docs_dir):
    with open(docs_dir + doc, 'r') as f:
        texts.append(f.read())

print(texts[1])

### `maps`
This key group allows designers to specify information about the starting state of the map, what information is contained and how it is grouped in the legend, and what viewports can be easily jumped to by the user.

## Common keys
- [`allowModification`](../common_keys/common_keys.md#allowModification)
- [`data`](../common_keys/common_keys.md#data)
- [`icon`](../common_keys/common_keys.md#icon)
- [`name`](../common_keys/common_keys.md#name)
- [`order`](../common_keys/common_keys.md#order)
- [`sendToApi`](../common_keys/common_keys.md#sendToApi)
- [`sendToClient`](../common_keys/common_keys.md#sendToClient)

## Special and custom keys
Key | Default | Description
--- | ------- | -----------
<a name="defaultViewport">`customMapKey.defaultViewport`</a> | | A dictionary object containing geo properties that set the map's default field of view. Also used by the "home" viewport button in the app.
`customMapKey.defaultViewport.bearing` | `0` | The initial bearing (rotation) of the m

In [3]:
#Tokenize docs
tokenizer_name = tiktoken.encoding_for_model('gpt-4')
tokenizer = tiktoken.get_encoding(tokenizer_name.name)

# create the length function
def tiktoken_len(text):
    tokens = tokenizer.encode(
        text,
        disallowed_special=()
    )
    return len(tokens)

print([tiktoken_len(doc) for doc in texts])

NameError: name 'texts' is not defined

In [58]:
texts[5]

'## Panes\nPanes are constructs primarily used to place UI controls (toggles, text and number fields, sliders, etc.), as well as buttons to allow interaction with actionable data. Custom panes can be designed to enable users to tune up the parameters of a simulation, navigate through different case study scenarios, reset the state of a simulation, synchronize data or settings with other users, and so on.\n\nPanes can be of different [`variant`](#pane-variant)s, so to keep the data structure examples simple and modular, you can examine each one at a time in the following switchables:\n\n<details>\n  <summary>App Settings pane</summary>\n\n```py\n"appSettings": {\n    "variant": "appSettings",\n},\n```\n</details>\n\n<details>\n  <summary>Options pane</summary>\n\n```py\n"customPropsPane": {\n    "name": "Example Props Pane",\n    "props": {\n        "numericHeader": {\n            "name": "Numeric Props",\n            "type": "head",\n            "help": "Some help for numeric props",\n

In [4]:
#intialize pinecone index
pinecone.init(api_key=pinecone_key, environment=pinecone_env)
pinecone.whoami()

index_name = 'gpt-4-embedded-docs'
print(pinecone.list_indexes())
# check if index already exists (it shouldn't if this is first time)
if index_name not in pinecone.list_indexes():
    # if does not exist, create index
    pinecone.create_index(
        index_name,
        dimension=dimensions,
        metric='cosine'
    )
    # wait for index to be initialized
    sleep(10)

# connect to index
index = pinecone.GRPCIndex(index_name)
# view index stats
index.describe_index_stats()


['gpt-4-embedded-docs']


{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 12}},
 'total_vector_count': 12}

In [42]:
# get embeddings
for idx, doc in enumerate(tqdm(texts)):
  # create embeddings (try-except added to avoid RateLimitError)
  try:
        res = openai.Embedding.create(input=doc, engine=embedding_model)
  except:
      done = False
      while not done:
          sleep(5)
          try:
              res = openai.Embedding.create(input=doc, engine=embedding_model)
              done = True
          except:
              pass

  embeds = [record['embedding'] for record in res['data']]
  to_upsert = list(zip([f'{idx}'], embeds))
  index.upsert(vectors=to_upsert)

100%|██████████| 12/12 [00:08<00:00,  1.47it/s]


In [66]:
# Example retrieval
query = "In the dashboard change the stat example a chart to group by sku"

res = openai.Embedding.create(
    input=[query],
    engine=embedding_model
)

# retrieve from Pinecone
xq = res['data'][0]['embedding']

# get relevant contexts (including the questions)
output = index.query(xq, top_k=3, include_metadata=True)

In [68]:
print(output)
print(len(texts))
print(texts[int(output['matches'][1]['id'])])

{'matches': [{'id': '11',
              'metadata': {},
              'score': 0.76675105,
              'sparse_values': {'indices': [], 'values': []},
              'values': []},
             {'id': '9',
              'metadata': {},
              'score': 0.7471639,
              'sparse_values': {'indices': [], 'values': []},
              'values': []},
             {'id': '7',
              'metadata': {},
              'score': 0.7391393,
              'sparse_values': {'indices': [], 'values': []},
              'values': []}],
 'namespace': ''}
11
# `dashboards`
The `dashboards` key allows API designers to create custom dashboards for displaying charts and tables of statistics and kpi data. This key allows for default information and layout to be specified, as well as whether the dashboards can be edited by the user.

## Special and custom keys
Key | Default | Description
--- | ------- | -----------
`customDashKey.dashboardLayout` | `[]` | A list of chart items (max of 4 item

In [23]:
state = ""
with open('state.json') as f:
    state = f.read()

print(state)

{"data":{"maps":{"data":{"map1":{"defaultViewport":{"longitude":-75.44766721108091,"latitude":40.34530681636297,"zoom":4.657916626867326,"pitch":0,"bearing":0,"height":1287,"altitude":1.5,"maxZoom":12,"minZoom":2},"optionalViewports":{"ov0":{"icon":"FaGlobeAsia","name":"Asia","zoom":4,"order":1,"pitch":0,"bearing":0,"maxZoom":12,"minZoom":2,"latitude":30,"longitude":121},"ov1":{"icon":"FaGlobeEurope","name":"EMEA","zoom":4,"order":1,"pitch":0,"bearing":0,"maxZoom":12,"minZoom":2,"latitude":47,"longitude":14}},"legendGroups":{"lga":{"name":"Legend Group A","nodes":{"nodeTypeA":{"value":true,"sizeBy":"numericPropExampleA","colorBy":"booleanPropExample","allowGrouping":true,"groupCalcByColor":"sum","groupCalcBySize":"mode"}},"arcs":{"T1":{"colorBy":"numericPropExampleA","sizeBy":"numericPropExampleB","value":true}},"order":1},"lgb":{"name":"Legend Group B","nodes":{"nodeTypeB":{"value":true,"sizeBy":"numericPropExampleB","colorBy":"booleanPropExample"}},"arcs":{"T2":{"colorBy":"numericPro

In [40]:
print(tiktoken_len(texts[int(output['matches'][0]['id'])]))
print(tiktoken_len(state))

663
9208


# Run Query through gpt4

Note: Currently using gpt-3.5-turbo-16k as 32k gpt4 hasn't been publicly released yet

In [70]:
api_model = "gpt-3.5-turbo-16k"

completion = openai.ChatCompletion.create(
  model=api_model,
  messages=[
    {"role": "system", "content": "Using the given documentation delimited by triple quotes and current api state encoded in JSON, respond to user requests with the path and value you would modify in the api to achieve the users desired action in the format \n```{\"path\": path, \"value\": value}```"},
    {"role": "user", "content": '"""' + texts[int(output['matches'][1]['id'])] + '""" \n\n' + state + '\n\n' + query},
  ]
)

print(completion.choices[0].message)


{
  "role": "assistant",
  "content": "{\"path\": \"dashboards.data.dash1.dashboardLayout.0.category\", \"value\": \"sku\"}"
}
